# Libbitcoin SPV Client: Watching a Wallet Address

In this example, we demonstrate how to subscribe to server updates pertaining to a specific payment address.

<br>
<img src="images/spv_watch_tx.jpg" alt="drawing" style="" width="800px"/>


### Libbitcoin-System (C++) Setup

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib")
#pragma cling load("bitcoin","bitcoin-protocol","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Chrono for subscription intervals
#include <chrono>

// Libbitcoin-System
// Libbitcoin-Protocol
#include <bitcoin/bitcoin.hpp> 
#include <bitcoin/protocol.hpp> 

<hr style="border: 0.5px dashed #000;">


## 1) Wallet Public Key Hash 

### 1.1) Decode public key hash from wallet address.

In [2]:
!bx address-decode -f json mk7gLaQfT3kEuZ3fAo3SivRkPWYgkFDfjH | jq '.wrapper.payload'

"3271bdf25c28ec5733b3faac3c3adb2b6fd0d15c"


In [3]:
auto my_pubkey_hash = bc::to_chunk(bc::base16_literal(
    "3271bdf25c28ec5733b3faac3c3adb2b6fd0d15c"));


## 2) Setup ZMQ Socket.

In [4]:
bc::protocol::zmq::context my_context(true); //started
bc::protocol::zmq::socket subscription_socket(
    my_context, bc::protocol::zmq::socket::role::dealer);

bc::code ec;
bc::config::endpoint public_endpoint("tcp://testnet2.libbitcoin.net:19091");
ec = subscription_socket.connect(public_endpoint);

## 3) Add ZMQ Poller.

* Updates can arrive at any time
* `socket.receive()` is blocking, so poller first checks if updates are queued.

In [5]:
// Adding our socket to our poller. Multiple Sockets can be added to poller.
bc::protocol::zmq::poller my_poller;
my_poller.add(subscription_socket);

// Polling for a specific socket requires the socket_id.
bc::protocol::zmq::identifier subscription_socket_id =
    subscription_socket.id();


## 4) Set Time for Subscription Renewal Intervals.

In [6]:
std::chrono::steady_clock::time_point next_subscribe_time = 
    std::chrono::steady_clock::now();

// Rewewal period (depends on server config)
auto renewal_period = std::chrono::minutes(1);


## 5) Renew Address Subscription and Poll for Address Updates.

* Address subscription request message:
    * `[-- "subscribe.address" --]`
    * `[--- 4-byte message id ---]`
    * `[----- public key hash ---]`
<br><br>

* Address subscription confirmation message:
    * `[---- "subscribe.address" ---]`
    * `[-- sent 4-byte message id --]`
    * `[----- 4-byte error code ----]`
<br><br>

* Address update reply message:
    * `[-------------------------- "notification.address" -------------------------]`
    * `[--------------------------- sent 4-byte message id ------------------------]`
    * `[-- 4-byte error code | 2-byte sequence | 4-byte height | 32-byte tx hash --]`
<br><br>

* Main loop accomplishes two objectives:
    * **Sends subscription renewal** on regular intervals.
    * **Polls socket for new address updates.**



In [ ]:
while (true)
{


    // Subscription Renewals.
    //----------------------------------------------------------------------

    std::chrono::steady_clock::time_point now_time
        = std::chrono::steady_clock::now();

    if (now_time >= next_subscribe_time)
    {
        // Subcription request message.
        bc::protocol::zmq::message my_request;
        std::string command = "subscribe.address";
        uint32_t message_id(0);
        my_request.enqueue(bc::to_chunk(command));
        my_request.enqueue(bc::to_chunk(
              bc::to_little_endian(message_id)));
        my_request.enqueue(my_pubkey_hash);
        
        // Socket send: Success/Failure
        if ((ec = my_request.send(subscription_socket)))
            {
            std::cout << ec.message() << std::endl;
            return 1;
            }

        // Set subscription rewewal period
        next_subscribe_time += renewal_period;
        message_id++;
    }


    // Polling Sockets for Subscription Confirmation / Updates.
    //----------------------------------------------------------------------

    bc::protocol::zmq::identifiers socket_ids = my_poller.wait(2000);

    if (socket_ids.contains(subscription_socket_id))
    {

       bc::protocol::zmq::message server_response;
       server_response.receive(subscription_socket);

       // Frame 0: Read the response command.
       std::string response_command = server_response.dequeue_text();
       // Frame 1: Read the message id (No checks implemented).
       uint32_t my_message_id;
       server_response.dequeue(my_message_id);
       // Frame 2: Read the payload.
       bc::data_chunk reply_payload;
       server_response.dequeue(reply_payload);

       // Payload[:4]: Read out error code
       bc::data_source reply_byte_stream(reply_payload);
       bc::istream_reader reply_byte_stream_reader(reply_byte_stream);
       bc::code message_ec = reply_byte_stream_reader.read_error_code();

       // Terminate if response message error.
       if (message_ec != bc::error::success )
           {
           std::cout << "Response Error" << std::endl;
           std::cout << message_ec.message() << std::endl;
           return 1;
           }

       if(response_command == "subscribe.address")
           {
           // subscription confirmed
           std::cout << "Subscription confirmed" << std::endl
                     << "---------------" << std::endl;
           }

       // Parse Update for block height & transaction hash
       //------------------------------------------------------------------

       else if(response_command == "notification.address")
       {

           // Update notification sequence.
           // Not implemented: Update sequence check.
           uint16_t update_sequence =
               reply_byte_stream_reader.read_2_bytes_little_endian();

           // Blockheight.
           uint32_t height =
               reply_byte_stream_reader.read_4_bytes_little_endian();

           // Transaction hash (little endian).
           // We use istream reader to reverse byte order.
           auto tx_hash_little_endian =
               reply_byte_stream_reader.read_bytes();
           bc::data_source tx_hash_byte_stream(tx_hash_little_endian);
           bc::istream_reader tx_hash_stream_reader(tx_hash_byte_stream);

           auto tx_hash =
               tx_hash_stream_reader.read_reverse<bc::hash_size>();

           std::cout
               << "Update received" << std::endl
               << "Notification sequence: "
               << update_sequence << std::endl
               << "Block height: "
               << height << std::endl
               << "Transaction hash: "
               << bc::encode_base16(bc::to_chunk(tx_hash)) << std::endl
               << "---------------" << std::endl;
        }
    }
}

Subscription confirmed
---------------
Update received
Notification sequence: 1
Block height: 0
Transaction hash: 79bbf6590c2d31e25d096212444a2b561a5bdc0cc7dd30636f9accab76d274ae
---------------
Subscription confirmed
---------------


## 6) Verify with Libbitcoin-Explorer.

In [ ]:
!bx watch-address mk7gLaQfT3kEuZ3fAo3SivRkPWYgkFDfjH